<img src="https://www.xoriant.com/cdn/ff/weqpbrtpXGjLpVQ_X-gWqsFlvjAxpv5Wv3xNW0A4vuQ/1602007254/public/2020-10/a-decisionTreesforClassification-AMachineLearningAlgorithm.jpg" width=600 height=600 />



# Decision Tree - Zoning

The goal of this script is to split a review into different sentences and then classify each of them into different predefined categories

<table>
  <tr>
    <th>Color</th>
    <th>Signification</th>
  </tr>
  <tr>
    <td>Purple</td>
    <td>Introduction</td>
  </tr>
  <tr>
    <td>Red</td>
    <td>Plot</td>
  </tr>
  <tr>
    <td>Blue</td>
    <td>General Analysis of the Play</td>
  </tr>
  <tr>
    <td>Green</td>
    <td>Visual, Auditory and Audible Details</td>
  </tr>
  <tr>
    <td>Orange</td>
    <td>Actors’ Performances</td>
  </tr>
  <tr>
    <td>Brown</td>
    <td>Remarks on the Structure of the Play</td>
  </tr>
  <tr>
    <td>Yellow</td>
    <td>Conclusion</td>
  </tr>
</table>

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics
from textblob import TextBlob
import text2emotion as te
import nltk
from gensim.summarization import keywords
from sklearn.tree import export_graphviz
from IPython.display import Image  
import pydotplus
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gaetanbrison/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gaetanbrison/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gaetanbrison/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/gaetanbrison/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
df = pd.read_excel("../Data/data_final_zoning_sentence_v1.xlsx")
df.head(3)

,Unnamed: 0,Sentence - Main_Sentence,Sentence - Women/Feminism,Sentence - LGBT / Queer / Sexuality / Gender,Sentence - Disabled,Sentence - Politics,Sentence - Supernatural,Sentence - Science,Sentence - Body,Sentence - Childhood,Sentence - Cultural difference / Race,Sentence - Death,Sentence - Education,Sentence - Family,Sentence - Environment,Sentence - Friendship,Sentence - Love,Sentence - Identity,Sentence - Memory,Sentence - Relationships,Sentence - Religion,Sentence - Violence,Sentence - Spectator_Cat,Sentence - Audience_Cat,Sentence - Show_Cat,Sentence - Story_Cat,Sentence - Decor_Cat,Sentence - Theatre_Cat,Sentence - Production_Cat,Sentence - Character_Cat,Sentence - End_Cat,Sentence - Beginning_Cat,Sentence - Max_Value,Sentence - Cluster,Sentence - Neg,Sentence - Neu,Sentence - Pos,Sentence - Compound,Sentence - Polarity,Sentence - Subjectivity,Sentence - Emotion,Sentence - Angry,Sentence - Fear,Sentence - Happy,Sentence - Sad,Sentence - Surpise,Sentence - Top_10_Keywords,Sentence - Count_Words,Sentence - Question_Mark,Sentence - Exclamation_Mark,Sentence - Word_Average,Sentence - ID,Sentence - Virgules_pct,Sentence - Point_virgules_pct,Sentence - Tirets_pct,Sentence - ID.1,Sentence - Declaratives_pct,Sentence - Interrogatives_pct,Sentence - Exclamatives_pct,Sentence - ID.2,Sentence - Nb_mots,Sentence - Adverbes,Sentence - Noms,Sentence - Verbes,Sentence - Adjectifs,Sentence - Superlatifs,Sentence - ID.3,Sentence - Passe,Sentence - Present,Sentence - Futur,Sentence - ID.4,Sentence - 1e_pers_s,Sentence - 2e_pers,Sentence - 3e_pers_s,Sentence - 1e_pers_p,Sentence - 3e_pers_p,Zone - Main_Sentence_x,Zone - Women/Feminism,Zone - LGBT / Queer / Sexuality / Gender,Zone - Disabled,Zone - Politics,Zone - Supernatural,Zone - Science,Zone - Body,Zone - Childhood,Zone - Cultural difference / Race,Zone - Death,Zone - Education,Zone - Family,Zone - Environment,Zone - Friendship,Zone - Love,Zone - Identity,Zone - Memory,Zone - Relationships,Zone - Religion,Zone - Violence,Zone - Spectator_Cat,Zone - Audience_Cat,Zone - Show_Cat,Zone - Story_Cat,Zone - Decor_Cat,Zone - Theatre_Cat,Zone - Production_Cat,Zone - Character_Cat,Zone - End_Cat,Zone - Beginning_Cat,Zone - Max_Value,Zone - Cluster,Zone - Neg,Zone - Neu,Zone - Pos,Zone - Compound,Zone - Polarity,Zone - Subjectivity,Zone - Emotion,Zone - Angry,Zone - Fear,Zone - Happy,Zone - Sad,Zone - Surpise,Zone - Top_10_Keywords,Zone - Count_Words,Zone - Question_Mark,Zone - Exclamation_Mark,Zone - Word_Average,Zone - Main_Sentence_y,Zone - Sentence_Type,Zone - Type_word_k,Zone - Type_word_v,Zone - ID,Zone - Virgules_pct,Zone - Point_virgules_pct,Zone - Tirets_pct,Zone - ID.1,Zone - Declaratives_pct,Zone - Interrogatives_pct,Zone - Exclamatives_pct,Zone - ID.2,Zone - Nb_mots,Zone - Adverbes,Zone - Noms,Zone - Verbes,Zone - Adjectifs,Zone - Superlatifs,Zone - ID.3,Zone - Passe,Zone - Present,Zone - Futur,Zone - ID.4,Zone - 1e_pers_s,Zone - 2e_pers,Zone - 3e_pers_s,Zone - 1e_pers_p,Zone - 3e_pers_p,Review - Blog,Review - Publication Date,Review - Reviewer,Review - Title of the Play,Review - Playwright,Review - Theatre,Review - Review with Zoning,Review - Intro-Purple,Review - Red-Plot,Review - Blue-General,Review - Green-Details,Review - Orange-Perf,Review - Borwn-Structure,Review - Yellow-Conclusion,Review - Latitude,Review - Longitude,Review - Women/Feminism,Review - LGBT / Queer / Sexuality / Gender,Review - Disabled,Review - Politics,Review - Supernatural,Review - Science,Review - Body,Review - Childhood,Review - Cultural difference / Race,Review - Death,Review - Education,Review - Family,Review - Environment,Review - Friendship,Review - Love,Review - Identity,Review - Memory,Review - Relationships,Review - Religion,Review - Violence,Review - Spectator_Cat,Review - Audience_Cat,Review - Show_Cat,Review - Story_Cat,Review - Decor_Cat,Review - Theatre_Cat,Review - Production_Cat,Review - Character_Cat,Review - End_Cat,Review - Beginning_Cat,Review - Max_Value,Review - Cluster,Review - Neg,Re

In [3]:
X = df[[
 'Sentence - Women/Feminism',
 'Sentence - LGBT / Queer / Sexuality / Gender',
 'Sentence - Disabled',
 'Sentence - Politics',
 'Sentence - Supernatural',
 'Sentence - Science',
 'Sentence - Body',
 'Sentence - Childhood',
 'Sentence - Cultural difference / Race',
 'Sentence - Death',
 'Sentence - Education',
 'Sentence - Family',
 'Sentence - Environment',
 'Sentence - Friendship',
 'Sentence - Love',
 'Sentence - Identity',
 'Sentence - Memory',
 'Sentence - Relationships',
 'Sentence - Religion',
 'Sentence - Violence',
 'Sentence - Spectator_Cat',
 'Sentence - Audience_Cat',
 'Sentence - Show_Cat',
 'Sentence - Story_Cat',
 'Sentence - Decor_Cat',
 'Sentence - Theatre_Cat',
 'Sentence - Production_Cat',
 'Sentence - Character_Cat',
 'Sentence - End_Cat',
 'Sentence - Beginning_Cat',
 'Sentence - Max_Value',
 'Sentence - Neg',
 'Sentence - Neu',
 'Sentence - Pos',
 'Sentence - Compound',
 'Sentence - Polarity',
 'Sentence - Subjectivity',
 'Sentence - Angry',
 'Sentence - Fear',
 'Sentence - Happy',
 'Sentence - Sad',
 'Sentence - Surpise',
 'Sentence - Count_Words',
 'Sentence - Question_Mark',
 'Sentence - Exclamation_Mark',
 'Sentence - Word_Average',
 'Sentence - Virgules_pct',
 'Sentence - Point_virgules_pct',
 'Sentence - Tirets_pct',
 'Sentence - Declaratives_pct',
 'Sentence - Interrogatives_pct',
 'Sentence - Exclamatives_pct',
 'Sentence - Nb_mots',
 'Sentence - Adverbes',
 'Sentence - Noms',
 'Sentence - Verbes',
 'Sentence - Adjectifs',
 'Sentence - Superlatifs',
 'Sentence - Passe',
 'Sentence - Present',
 'Sentence - Futur',
 'Sentence - 1e_pers_s',
 'Sentence - 2e_pers',
 'Sentence - 3e_pers_s',
 'Sentence - 1e_pers_p',
 'Sentence - 3e_pers_p',
 'Zone - Women/Feminism',
 'Zone - LGBT / Queer / Sexuality / Gender',
 'Zone - Disabled',
 'Zone - Politics',
 'Zone - Supernatural',
 'Zone - Science',
 'Zone - Body',
 'Zone - Childhood',
 'Zone - Cultural difference / Race',
 'Zone - Death',
 'Zone - Education',
 'Zone - Family',
 'Zone - Environment',
 'Zone - Friendship',
 'Zone - Love',
 'Zone - Identity',
 'Zone - Memory',
 'Zone - Relationships',
 'Zone - Religion',
 'Zone - Violence',
 'Zone - Spectator_Cat',
 'Zone - Audience_Cat',
 'Zone - Show_Cat',
 'Zone - Story_Cat',
 'Zone - Decor_Cat',
 'Zone - Theatre_Cat',
 'Zone - Production_Cat',
 'Zone - Character_Cat',
 'Zone - End_Cat',
 'Zone - Beginning_Cat',
 'Zone - Max_Value',
 'Zone - Neg',
 'Zone - Neu',
 'Zone - Pos',
 'Zone - Compound',
 'Zone - Polarity',
 'Zone - Subjectivity',
 'Zone - Angry',
 'Zone - Fear',
 'Zone - Happy',
 'Zone - Sad',
 'Zone - Surpise',
 'Zone - Count_Words',
 'Zone - Question_Mark',
 'Zone - Exclamation_Mark',
 'Zone - Word_Average',
 'Zone - Virgules_pct',
 'Zone - Point_virgules_pct',
 'Zone - Tirets_pct',
 'Zone - Declaratives_pct',
 'Zone - Interrogatives_pct',
 'Zone - Exclamatives_pct',
 'Zone - Nb_mots',
 'Zone - Adverbes',
 'Zone - Noms',
 'Zone - Verbes',
 'Zone - Adjectifs',
 'Zone - Superlatifs',
 'Zone - Passe',
 'Zone - Present',
 'Zone - Futur',
 'Zone - 1e_pers_s',
 'Zone - 2e_pers',
 'Zone - 3e_pers_s',
 'Zone - 1e_pers_p',
 'Zone - 3e_pers_p',
 'Review - Latitude',
 'Review - Longitude',
 'Review - Women/Feminism',
 'Review - LGBT / Queer / Sexuality / Gender',
 'Review - Disabled',
 'Review - Politics',
 'Review - Supernatural',
 'Review - Science',
 'Review - Body',
 'Review - Childhood',
 'Review - Cultural difference / Race',
 'Review - Death',
 'Review - Education',
 'Review - Family',
 'Review - Environment',
 'Review - Friendship',
 'Review - Love',
 'Review - Identity',
 'Review - Memory',
 'Review - Relationships',
 'Review - Religion',
 'Review - Violence',
 'Review - Spectator_Cat',
 'Review - Audience_Cat',
 'Review - Show_Cat',
 'Review - Story_Cat',
 'Review - Decor_Cat',
 'Review - Theatre_Cat',
 'Review - Production_Cat',
 'Review - Character_Cat',
 'Review - End_Cat',
 'Review - Beginning_Cat',
 'Review - Max_Value',
 'Review - Neg',
 'Review - Neu',
 'Review - Pos',
 'Review - Compound',
 'Review - Polarity',
 'Review - Subjectivity',
 'Review - Angry',
 'Review - Fear',
 'Review - Happy',
 'Review - Sad',
 'Review - Surpise',
 'Review - Nbr_of_Sentences',
 'Review - Length_Start_Count_Words_1',
 'Review - Length_End_Count_Words_1',
 'Review - Length_Start_Count_Words_2',
 'Review - Length_End_Count_Words_2',
 'Review - Length_Start_Count_Words_3',
 'Review - Length_End_Count_Words_3',
 'Review - Question_Mark_Start_1',
 'Review - Question_Mark_End_1',
 'Review - Question_Mark_Start_2',
 'Review - Question_Mark_End_2',
 'Review - Question_Mark_Start_3',
 'Review - Question_Mark_End_3',
 'Review - Exclamation_Mark_Start_1',
 'Review - Exclamation_Mark_End_1',
 'Review - Exclamation_Mark_Start_2',
 'Review - Exclamation_Mark_End_2',
 'Review - Exclamation_Mark_Start_3',
 'Review - Exclamation_Mark_End_3',
 'Review - Word_Average_Start_1',
 'Review - Word_Average_End_1',
 'Review - Word_Average_Start_2',
 'Review - Word_Average_End_2',
 'Review - Word_Average_Start_3',
 'Review - Word_Average_End_3',
 'Zone - Start Index',
 'Zone - Count Character',
 'Review - Count Character',
 'Zone - Count Word',
 'Review - Count Word',
 'Sentence - Start Index',
 'Sentence - Count Character',
 'Sentence - Count Word']]

In [4]:
y = df[["Target_Variable_Sentence_Type"]]

In [5]:
X = X.fillna(df.mean())

In [6]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

In [7]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [8]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8717948717948718


In [9]:
labels = ["Purple - Introduction","Red - Plot","Blue - General","Green - Details","Orange - Performances","Brown - Structure","Yellow - Conclusion"]

In [10]:
# Model performance on training set
y_pred_train =clf.predict(X_train)


accuracy = metrics.accuracy_score(y_train, y_pred_train)
print("Accuracy: {:.2f}".format(accuracy))
cm=confusion_matrix(y_train,y_pred_train)
print('Confusion Matrix: \n', cm)
print(classification_report(y_train, y_pred_train, target_names=labels))

Accuracy: 1.00
Confusion Matrix: 
 [[114   0   0   0   0   0   0]
 [  0 135   0   0   0   0   0]
 [  0   0  64   0   0   0   0]
 [  0   0   0  39   0   0   0]
 [  0   0   0   0  42   0   0]
 [  0   0   0   0   0  14   0]
 [  0   0   0   0   0   0  47]]
                       precision    recall  f1-score   support

Purple - Introduction       1.00      1.00      1.00       114
           Red - Plot       1.00      1.00      1.00       135
       Blue - General       1.00      1.00      1.00        64
      Green - Details       1.00      1.00      1.00        39
Orange - Performances       1.00      1.00      1.00        42
    Brown - Structure       1.00      1.00      1.00        14
  Yellow - Conclusion       1.00      1.00      1.00        47

             accuracy                           1.00       455
            macro avg       1.00      1.00      1.00       455
         weighted avg       1.00      1.00      1.00       455



In [11]:
# Predicting the test results
y_pred=clf.predict(X_test)

# Classification results on test set
from sklearn import metrics
accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}".format(accuracy))

from sklearn.metrics import confusion_matrix, classification_report
cm=confusion_matrix(y_test,y_pred)
print('Confusion Matrix: \n', cm)
print(classification_report(y_test, y_pred, target_names=labels))

Accuracy: 0.87
Confusion Matrix: 
 [[45  2  0  0  0  0  0]
 [ 1 42  1  0  0  0  0]
 [ 0  2 38  0  0  1  1]
 [ 0  1  1 10  1  1  0]
 [ 0  0  0  1 13  1  5]
 [ 0  1  0  1  0  5  0]
 [ 0  0  0  0  4  0 17]]
                       precision    recall  f1-score   support

Purple - Introduction       0.98      0.96      0.97        47
           Red - Plot       0.88      0.95      0.91        44
       Blue - General       0.95      0.90      0.93        42
      Green - Details       0.83      0.71      0.77        14
Orange - Performances       0.72      0.65      0.68        20
    Brown - Structure       0.62      0.71      0.67         7
  Yellow - Conclusion       0.74      0.81      0.77        21

             accuracy                           0.87       195
            macro avg       0.82      0.81      0.81       195
         weighted avg       0.87      0.87      0.87       195



In [12]:
import pickle
filename = 'finalized_model_tree__v3_sentence.sav'
pickle.dump(clf, open(filename, 'wb'))